<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#A-simulation-with-WOFOST" data-toc-modified-id="A-simulation-with-WOFOST-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>A simulation with WOFOST</a></span><ul class="toc-item"><li><span><a href="#Your-first-WOFOST-run" data-toc-modified-id="Your-first-WOFOST-run-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Your first WOFOST run</a></span></li></ul></li></ul></div>

<div style="float:right">
<table>
<tr>
    <td> <img src="figs/nceo_logo.png" alt="Drawing" style="width:200px;height:40px;"/> </td>
    <td> <img src="figs/gssti_logo.png" alt="Drawing" style="width:200px;height:40px;"/> </td>
    <td> <img src="figs/multiply_logo.png" alt="Drawing" style="width:40px;height:40px;"/> </td>
</tr>
    </table>
</div>

# The WOFOST crop model

**Author:** Jose Gómez-Dans (NCEO & UCL)  `j.gomez-dans@ucl.ac.uk`

In [7]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys, os
import copy
import datetime as dt
import pcse
from pcse.models import Wofost71_WLP_FD
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
print("This notebook was built with:")
print("python version: %s " % sys.version)
print("PCSE version: %s" %  pcse.__version__)
import datetime

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from wofost_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
This notebook was built with:
python version: 3.6.7 | packaged by conda-forge | (default, Jul  2 2019, 02:18:42) 
[GCC 7.3.0] 
PCSE version: 5.4.2


## Introduction

The WOFOST crop model has been around for a long time. It is widely used in both research and operational applications, and is fairly easy to use. It also has been used to simulate a large number of crops, and there are a number of extensions that deal with other crops.

While WOFOST lacks some of the more complex processes that more sophisticated models provide, it's more than adequate to familiarise oneself with a crop model.

The following diagram shows what WOFOST does

![schematic of the WOFOST model](figs/wofost_schematic.png)

Some of those boxes will already be familiar to you. We can see at the top that WOFOST is driven by radiation intercepted by the canopy. The canopy interception itself is not driven by a satellite fAPAR product as we saw earlier, but rather uses the predictions of LAI from the previous time step. Once photosynthesis is calculated and both growth and maintenance respiration are lost, partitioning of the assimilated C to the different plant organs takes place. These organs are:

1. Leaves
2. Stems
3. Roots
4. Storage organs (e.g. grain!)

A number of processes deal with the age and eventual death of leaves, stems and roots.

WOFOST implements a large number of processes to model a crop. Although not shown here, it does have a soil moisture module, and many of the processes are controlled by the development stage of the crop (its *phenology*). The phenology is simulated often by thermal time (there are other options, using e.g. daylength and stuff).

WOFOST doesn't implement many processes that are to do with management, pests, complex soil chemistry, etc. However, some management and additional information is needed to run the model, such as selecting the crop and variety, providing meteo data, configuring things around... This is all done for you (lucky you!). Since our aim here is to build towards linking the model with EO data, we'll consider some parameters that are important to diagnose what's happening. These will include LAI, soil moisture, total storage organ weight, and the development stage. 


## A simulation with WOFOST

There are two ways of using the model: potential and water limited. the former ignores the effect of water stress, whereas the latter does some soil water calculations and includes the effect of soil moisture in crop development. 

A simulation starts by the user selecting a soil type, a crop variety, some meteorological inputs, and some other parameters such as either the sowing or emergence date, the amount of water in the soil at the start of the simulation, etc. Once this is done, the model takes over, calculating soil hydraulics and any limitations to growth due to lack of soil water, and calculating radiation interception, photosynthesis, removing respiration losses and eventually partioning the assimilate. Defined by two crop variety parameters, the model calculates the thermal time between emergence and flowering, and between flowering and harvest, this is the development stage (DVS), which goes from 0 (emergence) to 1 (flowering) to 2 (harvest). Many processes are impacted by DVS.

### Your first WOFOST run

The next grid will run WOFOST and plot some graphs of some state variables with respect to time as a function of sowing and harvest date. 

In [13]:
change_sowing_slider();

interactive(children=(DatePicker(value=datetime.date(2011, 7, 1), description='sowing_date'), DatePicker(value…

In [ ]:
debug

> /home/ucfajlg/.local/lib/python3.6/site-packages/pcse/base/weather.py(159)__setattr__()
    157                     msg = "Value (%s) for meteo variable '%s' outside allowed range (%s, %s)." % (
    158                     value, key, vmin, vmax)
--> 159                     raise exc.PCSEError(msg)
    160         SlotPickleMixin.__setattr__(self, key, value)
    161 

ipdb> up
> /home/ucfajlg/.local/lib/python3.6/site-packages/pcse/base/weather.py(130)__init__()
    128             except (KeyError, ValueError, TypeError) as e:
    129                 msg = "%s: Weather attribute '%s' missing or invalid numerical value: %s"
--> 130                 logging.warning(msg, self.DAY, varname, value)
    131 
    132         # Loop over optional arguments

ipdb> p key
*** NameError: name 'key' is not defined
ipdb> p self.DAY
datetime.date(2011, 2, 5)
